<a href="https://colab.research.google.com/github/juliojesus15/SankeyDiagramMapping/blob/main/MappingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Utils**

In [ ]:
import json

def get_link_status(source, target):
  results = { "0": "dropout_semester", "1": "repeat_semester", "2": "pass_semester" }

  if source == "1N":
    return results.get(target)
  elif target == "0":
    return results.get("0")
  elif source == target:
    return results.get("1")
  elif source < target:
    return results.get("2")

def intersect_arrays(array1, array2):
  output = set(array1) & set(array2)
  output = list(output)
  found = True if len(output) > 0 else False
  return found, output

def save_dict_to_json(filename, data):
  with open(filename, "w") as archivo:
    json.dump(data, archivo)
  output = json.dumps(data)

def read_json(filename):
  with open(filename, "r") as archivo:
    datos_json = json.load(archivo)
  return datos_json


In [ ]:
# Pruebas
a = {'6f4005fb-f844-457a-838e-0f33ddb82c2f': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02']}, '8e7c7ae4-5e46-4f51-814e-8e9b2d348507': {'roadmap': [1, 2, 3, 4, 4, 5, 6, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02']}, 'a14e1a50-2838-4d86-9f71-46e90cd16710': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02', '2020-01']}, '5bf4e1b9-5322-4db4-b2d9-861ff1b8effa': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02', '2020-01']}, 'c8777913-e059-45fb-acef-4dcb1a7060f0': {'roadmap': [1, 1, 2, 2, 2, 3, 3, 3, 3, 3], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02']}, 'e013ef3a-db3e-4127-a03d-1a8e206ded36': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02', '2020-01']}, 'a5907af9-37ae-4a88-92e5-2dc2bb68ce55': {'roadmap': [1, 2, 2, 3, 4, 5, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01']}, '35502942-ed1f-4e57-9aae-1a82f470182b': {'roadmap': [1, 2, 2, 2, 3, 3, 3, 4, 4, 4], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02']}, '4fe159df-b897-44bc-93d7-996816ae6e09': {'roadmap': [1, 1, 2, 3, 4, 5, 6, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02']}, '0efa1193-d981-4041-8f61-2bcdd9bbaaf3': {'roadmap': [1, 2, 3, 4, 4, 5, 5, 5, 5, 5], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02']}, '60b567bd-0922-4e3c-b1c1-293d5f7d3968': {'roadmap': [1, 1, 2, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02']}}
b = {'6f4005fb-f844-457a-838e-0f33ddb82c2f': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02']}, '8e7c7ae4-5e46-4f51-814e-8e9b2d348507': {'roadmap': [1, 2, 3, 4, 4, 5, 6, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02']}, 'a14e1a50-2838-4d86-9f71-46e90cd16710': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02', '2020-01']}, '5bf4e1b9-5322-4db4-b2d9-861ff1b8effa': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02', '2020-01']}, 'c8777913-e059-45fb-acef-4dcb1a7060f0': {'roadmap': [1, 1, 2, 2, 2, 3, 3, 3, 3, 3], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02']}, 'e013ef3a-db3e-4127-a03d-1a8e206ded36': {'roadmap': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02', '2020-01']}, 'a5907af9-37ae-4a88-92e5-2dc2bb68ce55': {'roadmap': [1, 2, 2, 3, 4, 5, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01']}, '35502942-ed1f-4e57-9aae-1a82f470182b': {'roadmap': [1, 2, 2, 2, 3, 3, 3, 4, 4, 4], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02']}, '4fe159df-b897-44bc-93d7-996816ae6e09': {'roadmap': [1, 1, 2, 3, 4, 5, 6, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02']}, '0efa1193-d981-4041-8f61-2bcdd9bbaaf3': {'roadmap': [1, 2, 3, 4, 4, 5, 5, 5, 5, 5], 'times': ['2015-01', '2015-02', '2016-01', '2016-02', '2017-01', '2017-02', '2018-01', '2018-02', '2019-01', '2019-02']}, '60b567bd-0922-4e3c-b1c1-293d5f7d3968': {'roadmap': [1, 1, 2, 0], 'times': ['2015-01', '2015-02', '2016-01', '2016-02']}}

test = {'a': a, "b": b }

for key in test:
  filename = "drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/raw/roadmap/" + key + ".json"
  data = test[ key ]
  save_dict_to_json(filename, data)

# **Dataset**

In [ ]:
import pandas as pd
import numpy as np

# Cargando datasets
students = pd.read_csv("drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/students.csv")
roadmap = pd.read_csv("drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/semesters.csv")

In [ ]:
students.head(3)

In [ ]:
roadmap.head(3)

In [ ]:
dataset = pd.merge(students, roadmap, on='student_id')
dataset.head(3)

# **Mapping Data**

In [ ]:
import numpy as np

def create_roadmap_container(dataframe):
  container = {}

  for index, row in dataframe.iterrows():
    student_id = row["student_id"]
    enrollment = row["enrollment"]
    container.setdefault(enrollment, {}).setdefault(student_id, [])

  return container

def student_roadmap_mapping(dataframe, container):
  for index, row in dataframe.iterrows():
    time = row["timelapse"]
    studentId = row["student_id"]
    semester = row["semester"]
    enrollment = row["enrollment"]

    roadmap = "ES" if len(container[ enrollment ][ studentId ]) == 0 else str(semester)
    container[enrollment][studentId].append({"semester": roadmap, "timelapse": time})

  return container

# Crea un diccionario vacío con las claves proporcionadas
def create_container(keys):
  return {key: {} for key in keys}

def student_mapping(dataframe, container):
  for register in dataframe.index:
    timelapse_id, enrollment, student_id, semester = dataframe.loc[register, ["timelapse", "enrollment", "student_id", "semester"]]

    # Verificando si es la primera matricula (N = new)
    key = f"{semester}N" if timelapse_id == enrollment else str(semester)

    student = {
      "id": student_id,
      "name": dataframe.loc[register, "name"],
      "lastname": dataframe.loc[register, "lastname"],
      "gender": dataframe.loc[register, "gender"],
      "enrollment": enrollment
    }

    container.setdefault(timelapse_id, {})
    container[timelapse_id].setdefault(key, {"semester_id": key, "student_ids": [], "students": {}})
    container[timelapse_id][key]['student_ids'].append(student_id)
    container[timelapse_id][key]['students'][student_id] = student

  return container


# **Constants**

In [ ]:
# sem_roadmap
node_neighborhood = {
  '0':  [],
  '1N': ['1', '2','0'],
  '1':  ['1', '2','0'],
  '2':  ['2', '3','0'],
  '3':  ['3', '4','0'],
  '4':  ['4', '5','0'],
  '5':  ['5', '6','0'],
  '6':  ['6', '7','0'],
  '7':  ['7', '8','0'],
  '8':  ['8', '9','0'],
  '9':  ['9', '10','0'],
  '10': ['10','11','0'],
  '11': []
}

# sem_properties
node_properties = {
    "1N" : { "name": "IN", "vertical_order": 0,  "status": "start"  },  # Nodo inicial para ingresantes
    "1"  : { "name": "1",  "vertical_order": 1,  "status": "regular"},  # Nodo para representar alumnos en el semestre 'X'
    "2"  : { "name": "2",  "vertical_order": 2,  "status": "refular"},
    "3"  : { "name": "3",  "vertical_order": 3,  "status": "regular"},
    "4"  : { "name": "4",  "vertical_order": 4,  "status": "regular"},
    "5"  : { "name": "5",  "vertical_order": 5,  "status": "regular"},
    "6"  : { "name": "6",  "vertical_order": 6,  "status": "regular"},
    "7"  : { "name": "7",  "vertical_order": 7,  "status": "regular"},
    "8"  : { "name": "8",  "vertical_order": 8,  "status": "regular"},
    "9"  : { "name": "9",  "vertical_order": 9,  "status": "regular"},
    "10" : { "name": "10", "vertical_order": 10, "status": "regular"},
    "11" : { "name": "GR", "vertical_order": 11, "status": "finish" },  # Nodo final, graduados
    "0"  : { "name": "DR", "vertical_order": 12, "status": "dropout"}   # Nodo para desertores
}


# **Layout Sankey Diagram**

In [ ]:
# set a especific timelapse (exmaple: 2010-01), a dict (nodes) to storage all nodes, and container with all data
def generate_nodes_by_timeId(timeId, nodes, container):
  # Check if timelapse exist in our dict, else create a timelapse as key
  if not (timeId in nodes):
    nodes = { **nodes, timeId: [] }

  semesters = container[ timeId ]

  for semester in semesters:
    nodeId = semester + "-" + timeId
    nodes[ timeId ].append({
      'nodeId': nodeId,
      'name':   node_properties[ semester ]["name"],
      'vertical_order':  node_properties[ semester ]["vertical_order"],
      'status': node_properties[ semester ]["status"],
    })

  return nodes

def generate_link_between_timesId(semester_from, semester_to, links, container):
  link_key = semester_from + "_" + semester_to

  if link_key not in links:
    links[link_key] = []

  source = container[ semester_from ]
  target = container[ semester_to ]


  for key in source:
    neighborhood = node_neighborhood[ key ]

    # Analizando el vecindario de cada semestre
    for neighbor in neighborhood:
      if (neighbor in target):
        in_common, student_ids = intersect_arrays(source[ key ]['student_ids'], target[ neighbor ]['student_ids'])
        if in_common:
          students = [container[semester_from][key]['students'][ids] for ids in student_ids]
          source_id = key + "-" + semester_from
          target_id = neighbor + "-" + semester_to

          # Cambiar
          value =  round((len(student_ids)*100)/len(source[ key ]['student_ids']), 2)
          status = get_link_status(key, neighbor)

          links[ link_key ].append({
            "source": source_id,
            "target": target_id,
            "value": value,
            "students": students,
            "status": status
          })
  return links


# **Main**

In [ ]:
timelapse_ids = dataset.timelapse.unique()
timelapse_ids = np.array(timelapse_ids)
timelapse_ids.sort()

#timelapse_ids

In [ ]:
# Creando diccionario vacío con los timelapse (2010-01 ...)
CONTAINER = create_container(timelapse_ids)

# Mapeando cada estudiante en el semestre que ha cursado
CONTAINER = student_mapping(dataset, CONTAINER)

# Creando un diccionario para mapear cada estudiante en el timelapse de ingreso
CONTAINER_ROADMAP = create_roadmap_container(students)

# Genera el roadmap de cada estudiante
CONTAINER_ROADMAP = student_roadmap_mapping(dataset, CONTAINER_ROADMAP)

In [ ]:
# Opcional
for register in CONTAINER:
  filename = "drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/raw/container/" + register + ".json"
  data = CONTAINER[ register ]
  #print(type(data))
  #save_dict_to_json(filename, data)

In [ ]:
# Roadmaps
for register in CONTAINER_ROADMAP:
  #print(register)
  filename = "drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/raw/roadmap/" + register + ".json"
  data = CONTAINER_ROADMAP[ register ]
  #save_dict_to_json(filename, data)


In [ ]:
CONTAINER_LINKS = {}

for index in range(0, len(timelapse_ids) -1):
  semester_from = timelapse_ids[ index ]
  semester_to = timelapse_ids[ index + 1 ]
  CONTAINER_LINKS = generate_link_between_timesId(semester_from, semester_to, CONTAINER_LINKS, CONTAINER)

#save_dict_to_json("links.json", CONTAINER_LINKS)

In [ ]:
for register in CONTAINER_LINKS:
  #print(register)
  filename = "drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/raw/links/" + register + ".json"
  data = CONTAINER_LINKS[ register ]
  #save_dict_to_json(filename, data)

In [ ]:
CONTAINER_NODES = {}

for time in timelapse_ids:
  CONTAINER_NODES = generate_nodes_by_timeId(time , CONTAINER_NODES, CONTAINER)

#save_dict_to_json("nodes.json", CONTAINER_NODES)

In [ ]:

for register in CONTAINER_NODES:
  #print(register)
  filename = "drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/raw/nodes/" + register + ".json"
  data = CONTAINER_NODES[ register ]
  #save_dict_to_json(filename, data)